> Ingresos por categoria de los productos y visualización de la jerarquías

In [60]:
# Librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Visualización interactiva
import plotly.express as px

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")

sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)

In [61]:
df_order_items = pd.read_pickle('../data/clean/order_items.pkl')

df_order_items = df_order_items[["product_id", "quantity", "unit_price", "line_total", "discount_amount"]]

df_order_items

,product_id,quantity,unit_price,line_total,discount_amount
order_item_id,,,,,
1,940377,6,569.28,"3,415.68",0.00
2,935931,2,"1,090.20",<NA>,0.00
3,905111,7,<NA>,"1,176.98",154.52
4,920065,3,464.45,"1,393.35",<NA>
5,927289,8,283.12,<NA>,<NA>
...,...,...,...,...,...
285239,912297,1,88.40,88.40,0.00
285240,945504,10,"1,435.85","14,358.50",0.00
285241,972378,8,172.43,"1,379.44",158.71


Vamos a empezar procesando rapidamente estos detalles de ordenes. Como criterios, vamos si tenemos cantidad y precio unitario vamos a calcular el precio nuevamente ya que a veces no coinciden, si no tenemos alguno de los dos, vamos a tomar el precio de la linea. Si no tenemos alguno de los primeros 2 ni el precio de la linea, vamos a descartar esa linea. Además si hay un descuento vamos a aplicarlo en el momento a modo de rebaja.

In [62]:
df_order_items['calculated_price'] = np.where(
    (df_order_items['quantity'].notna()) & (df_order_items['unit_price'].notna()),
    df_order_items['quantity'] * df_order_items['unit_price'],
    np.where(
        df_order_items['line_total'].notna(),
        df_order_items['line_total'],
        np.nan
    )
).copy()

df_order_items = df_order_items.dropna(subset=['calculated_price'])
df_order_items['final_price'] = df_order_items['calculated_price'] - df_order_items['discount_amount'].fillna(0)

df_priced_order_items = df_order_items[["product_id", "final_price"]].copy()

df_priced_order_items

/tmp/ipykernel_7824/739296204.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order_items['final_price'] = df_order_items['calculated_price'] - df_order_items['discount_amount'].fillna(0)


,product_id,final_price
order_item_id,,
1,940377,"3,415.68"
2,935931,"2,180.40"
3,905111,"1,022.46"
4,920065,"1,393.35"
5,927289,"2,264.96"
...,...,...
285239,912297,88.40
285240,945504,"14,358.50"
285241,972378,"1,220.73"


Ahora vamos a agrupar por producto para obtener el total vendido por producto.

In [63]:
grouped_by_product = df_priced_order_items.groupby('product_id').agg(
    total_revenue=pd.NamedAgg(column='final_price', aggfunc='sum'),
)

grouped_by_product.index = grouped_by_product.index.astype(str)

grouped_by_product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93320 entries, 900019 to 1000000
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   total_revenue  93320 non-null  Float64
dtypes: Float64(1)
memory usage: 1.5+ MB


El siguiente paso, sería mapear todos estos productos a sus categorías y aplicar nuevamente la agrupación esta vez por categoría para obtener el revenue por categoría. Para ellos vamos a import el dataset de productos y vamos a hacer un merge con el dataset que tenemos de ventas por producto mantendremos por ahora las ventas que no matcheen con ningún producto o los productos que no tengan categoría como NA.

In [64]:
products_df = pd.read_pickle('../data/clean/products.pkl')

products_df = products_df[["category_id"]].copy()

products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 1 to 1000000
Data columns (total 1 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   category_id  1000000 non-null  Int64
dtypes: Int64(1)
memory usage: 16.2+ MB


In [65]:
revenue_by_product_and_category = grouped_by_product.merge(
    products_df,
    left_index=True,
    right_index=True,
    how='left'
)

revenue_by_product_and_category.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93320 entries, 900019 to 1000000
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   total_revenue  93320 non-null  Float64
 1   category_id    93320 non-null  Int64  
dtypes: Float64(1), Int64(1)
memory usage: 4.3+ MB


In [66]:
grouped_by_categories = revenue_by_product_and_category.groupby('category_id').agg(
    total_revenue=pd.NamedAgg(column='total_revenue', aggfunc='sum'),
)

grouped_by_categories.describe()

,total_revenue
count,179.00
mean,"4,007,056.32"
std,"2,166,763.05"
min,"197,788.63"
25%,"3,870,623.00"
50%,"4,152,377.55"
75%,"4,314,727.37"
max,"13,083,384.12"


Ya tenemos el revenue por categoría en función de nuestras ordenes. Lo siguiente que podemos hacer es mostrar las 5 categorías con más ingresos de nuestro dataset. Para ello primero necesitamos saber el nombre de la categoría. Vamos a importar el dataset de categorías y hacer un merge nuevamente con el dataset que tenemos de ventas por categoría. Nos guardaremos la categoría padre para luego poder mostrar la jerarquía.

In [67]:
categories_with_parent = pd.read_pickle('../data/clean/categories.pkl')

categories_with_parent = categories_with_parent[["name", "parent_category_id"]].copy()
categories_with_parent

,name,parent_category_id
id,,
1,Smartphones,180
2,Laptops,180
3,Tablets,<NA>
4,Cameras,<NA>
5,Televisions,180
...,...,...
204,Collectibles,<NA>
205,Tickets & Experiences,<NA>
206,Musical Instruments,<NA>


In [80]:
revenue_by_category = grouped_by_categories.merge(
    categories_with_parent,
    left_index=True,
    right_index=True,
    how='outer'
).fillna({'total_revenue': 0})

revenue_by_category = revenue_by_category[["name", "parent_category_id", "total_revenue"]].copy()

revenue_by_category

,name,parent_category_id,total_revenue
1,Smartphones,180,"12,277,289.38"
2,Laptops,180,"12,650,412.70"
3,Tablets,<NA>,"13,083,384.12"
4,Cameras,<NA>,"12,916,107.50"
5,Televisions,180,"12,043,372.85"
...,...,...,...
204,Collectibles,<NA>,0.00
205,Tickets & Experiences,<NA>,0.00
206,Musical Instruments,<NA>,0.00
207,Games & Virtual Goods,<NA>,0.00


In [81]:
TOP_N = 5

sorted_categories = revenue_by_category.sort_values(by='total_revenue', ascending=False)
top_categories = sorted_categories.head(TOP_N)

top_categories

,name,parent_category_id,total_revenue
3,Tablets,<NA>,"13,083,384.12"
4,Cameras,<NA>,"12,916,107.50"
2,Laptops,180,"12,650,412.70"
6,Headphones,180,"12,389,139.51"
1,Smartphones,180,"12,277,289.38"


Esto nos indica que las Tablets individualmente son las categorías con más revenue pero no sabemos exactamente si hay alguna categoría padre que contenga varias y que en conjunto superen a las Tablets o a los otros. Vamos a hacer un recorrido por la jerarquía para cada una de las categorías y ver si alguna categoría padre tiene más revenue que las tablets.

In [106]:
df = revenue_by_category.copy().reset_index()

df["direct_revenue"] = df["total_revenue"]
df = df.rename(columns={"index": "category_id"})

children = df.groupby("parent_category_id")["category_id"].apply(list).to_dict()

# Función recursiva para acumular revenue
def accumulate(cat_id):
    row = df.loc[df["category_id"] == cat_id].iloc[0]
    total = row["total_revenue"]
    if cat_id in children:  # si tiene hijos
        for child in children[cat_id]:
            total += accumulate(child)
    df.loc[df["category_id"] == cat_id, "total_revenue"] = total
    return total

# Aplicar a las raíces (categorías sin parent_id)
roots = df.loc[df["parent_category_id"].isna(), "category_id"]
for r in roots:
    accumulate(r)

total_revenue_by_category = df
total_revenue_by_category

,category_id,name,parent_category_id,total_revenue,direct_revenue
0,1,Smartphones,180,"12,277,289.38","12,277,289.38"
1,2,Laptops,180,"12,650,412.70","12,650,412.70"
2,3,Tablets,<NA>,"13,083,384.12","13,083,384.12"
3,4,Cameras,<NA>,"12,916,107.50","12,916,107.50"
4,5,Televisions,180,"12,043,372.85","12,043,372.85"
...,...,...,...,...,...
203,204,Collectibles,<NA>,"25,166,757.78",0.00
204,205,Tickets & Experiences,<NA>,"24,766,040.32",0.00
205,206,Musical Instruments,<NA>,"31,424,855.78",0.00
206,207,Games & Virtual Goods,<NA>,"25,220,420.16",0.00


Ahora podemos ver que aparecen categorías que antes ni aparecían ya que tienen todas las ventas de sus categorías hijas. Veamos ahora el top 5 de categorías con más revenue.

In [107]:
sorted_total_revenue = total_revenue_by_category.sort_values(by='total_revenue', ascending=False)

sorted_total_revenue.head(TOP_N)

,category_id,name,parent_category_id,total_revenue,direct_revenue
179,180,Electronics,<NA>,"49,360,214.44",0.00
186,187,Automotive,<NA>,"40,399,254.61",0.00
17,18,Furniture,<NA>,"37,673,418.19","4,199,633.68"
205,206,Musical Instruments,<NA>,"31,424,855.78",0.00
197,198,Kitchen & Dining,<NA>,"29,457,665.77",0.00


Ahora las categorías son totalmente otras... Veamos la información en forma de treemap para entender mejor como se distribuye el revenue por categorías y subcategorías.

In [173]:
df = total_revenue_by_category.copy()

# Asegurar tipos string para ids/parents (evita conflictos None/NaN)
df["id_str"] = df["category_id"].astype(str)
df["parent_str"] = df["parent_category_id"].astype("string")

# Treemap jerárquico con drill-down
fig = px.treemap(
    df,
    ids="id_str",
    names="name",
    parents="parent_str",
    values="total_revenue",
    color="total_revenue",
    color_continuous_scale="Greens",
)

fig.update_traces(
    # Texto dentro de cada rectángulo
    texttemplate="<b>%{label}</b><br>%{value:,.0f}",
    textinfo="label+value+percent entry",
    # Hover con % relativo al padre
    hovertemplate=(
        "<b>%{label}</b><br>"
        "Revenue total: %{value:,.0f}<br>"
        "Porcentaje: %{percentParent:.1%}<extra></extra>"
    ),
    # Apariencia
    root_color="lightgray",
    tiling=dict(pad=2, squarifyratio=1.2),
    # Pathbar (migas de pan) arriba para navegar
    pathbar=dict(visible=True, textfont=dict(size=12)),
)

fig.update_layout(
    title=dict(
        text="Categorías principales por revenue total",
        x=0.5,
        font=dict(size=20),
    ),
    coloraxis_colorbar=dict(title="Revenue total", tickformat=",.0f"),
    margin=dict(t=60, r=10, b=10, l=10),
)

fig.show()

Este gráfico nos muestra claramente como se distribuye el revenue por categorías incluyendo la revenue de las subcategorías, es interactivo y consistente con todo aquello que fuimos calculando.